# Introduction to TensorFlow (ver 1.8.0)

TensorFlow is a powerful open source software library for numerical computation, particularly well suited and find-tuned for large-scale Machine Learning.

Its basic principle is simple:
 - You first define in Python a graph of computation to perform
 - And then Tensorflow takes that graph and run it efficiently using optimized C++ code
 
See the Detail:
 - https://github.com/jtoy/awesome-tensorflow

In [3]:
import tensorflow as tf

C:\Users\p\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


- **tf.reset_default_graph()**
    - Clears the default graph stack and resets the global default graph.


In [4]:
tf.reset_default_graph() # 메모리에 올라간 데이터 제거

x = tf.Variable(3, name = "x")
y = tf.Variable(4, name = "y")

In [5]:
f = x*x*y + y + 2
# ? = 3*3*4 + 4 + 2

In [6]:
f #?????????????????

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [7]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print (result)

42


After sess.run, You MUST run below code.

In [8]:
sess.close()

In [9]:
sess.run(f)
## RuntimeError will be occur.

RuntimeError: Attempted to use a closed Session.

- Method 2: with tf.Session() as sess:

In python the with keyword is used when working with **unmanaged resources** (like file streams). It is similar to the using statement in VB.NET and C#. It allows you to ensure that a resource is **"cleaned up"** when the code that uses it finishes running, even if exceptions are thrown. It provides 'syntactic sugar' for try/finally blocks.

In [ ]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [ ]:
result

In [ ]:
sess.run(f)
## RuntimeError will be occur.

- Method 2-1: tf.global_variables_initializer()

In [9]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

In [10]:
result

42

- InteractiveSession()
    - the only difference from a regular Session is that when an InteractiveSession is created it automatically sets itself as the default session, so you don't need a with block.

In [11]:
init = tf.global_variables_initializer()

sess = tf.InteractiveSession()
init.run()
result = f.eval()

In [12]:
result

42

In [13]:
sess.run(f)

42

In [14]:
sess.close()

## Linear Regression

In [15]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

### Using the Normal Equation

In [16]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing

reset_graph()

housing = fetch_california_housing()

In [17]:
print(housing.DESCR)

California housing dataset.

The original database is available from StatLib

    http://lib.stat.cmu.edu/datasets/

The data contains 20,640 observations on 9 variables.

This dataset contains the average house value as target variable
and the following input variables (features): average income,
housing average age, average rooms, average bedrooms, population,
average occupation, latitude, and longitude in that order.

References
----------

Pace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions,
Statistics and Probability Letters, 33 (1997) 291-297.




In [18]:
print (housing.feature_names)
print (housing.data[0])
print (housing.target[0])

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
[   8.3252       41.            6.98412698    1.02380952  322.
    2.55555556   37.88       -122.23      ]
4.526


### Normal Equation

$$\theta = (X^T X)^{-1} X^T y$$
where, $X$, design matrix as a matrix of $m$ rows


In [21]:
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    print (theta_value)

[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


compare with pure numpy

In [22]:
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(theta_numpy)

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654266e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


Compare with Scikit-Learn

In [23]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)
